<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/01-트랜스포머_소개.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  트랜스포머 소개 목차
* [Chapter 1 인코더-디코더 프레임워크](#chapter1)
* [Chapter 2 어텐션 메커니즘](#chapter2)
* [Chapter 3 NLP의 전이 학습](#chapter3)
* [Chapter 4 허깅페이스 트랜스포머스](#chapter4)
* [Chapter 5 트랜스포머 애플리케이션 둘러보기](#chapter5)
* [Chapter 6 허깅페이스 생태계](#chapter6)

## Chapter 1 인코더-디코더 프레임워크 <a class="anchor" id="chapter1"></a>
1. RNN은 단어 또는 문자와 같은 입력을 받아 네트워크를 통화시킨 후 은닉 상태라는 벡터를 출력한다.
    - 출력된 정보를 피드백 루프로 보내 자기 자신에 입력으로 사용한다.
    - RNN은 출력한 정보의 일부를 다음 스텝에 사용하게 된다.
    - 이런 구조는 NLP 작업, 음성 처리, 시계열 작업에 널리 사용되고있다.

        ![RNN](image/01_rnn.png)

2. RNN은 단어 시퀸스를 한 언너에서 다른 언어로 매핑하는 기계번역 시스템을 개발할 때 중요한 역활을 했다.
    - 인코더-디코더, 시퀸스-투-시퀸스 구조로 처리되며 입렬과 출력이 임의의 길이를 가진 시퀸스일 때 잘 맞는다.
    - 인코더는  입력 시킨스의 정보를 마지막 은닉 상태라고 부르는 수치 표현으로 인코딩된 후 디코더로 전달되어 출력 시퀸스가 생성된다.
    - "Transformer are great!"를 디코딩해 독일어 "Transformer sind großartig!"로 변환하는 예시를 보자.
    - 입력 단어는 순차적으로 인코더에 주입되고, 출력 단어는 위에서 아래 방향으로 한 번에 하나씩 생성된다.

        ![RNN_Encoder-Decoder](image/01_rnn_encoder_decoder.png)

3. 위의 구조는 인코더의 마지가 은닉 상태가 정보 병목이 된다는 약점이 있다.
    - 정보 병목: 입력 시퀸스의 모든 정보가 고정된 크기의 벡터로 압축되어야 하는 문제
    - 디코더는 인코더의 마지막 은닉 상태만을 참조해 출력을 만들므로 여기에 전체 입력 시퀸스에 대한 정보가 담겨 있어야 한다.
    - 긴 문장의 경우 정보 손실이 발생할 수 있고, 모든 것을 고정된 하나의 표현으로 압축하는 과정의 시작부터 정보가 손실될 수 있다.
    - 이를 해결하기 위해 디코더가 모든 은닉 상태에 접근해 이 병목을 제거하는데 이를 어텐션 메커니즘이라 부른다.


## Chapter 2 어텐션 메커니즘 <a class="anchor" id="chapter2"></a>
1. 어텐션은 입력 시퀸스에서 은닉 상태를 만들지 않고 스탭마다 인코더에서 디코더가 참조할 은닉 상태를 출력한다는 아이디어에서 출발한다.
    - 모든 상태를 동시에 사용하려면 디코더에 많은 입력이 발생하므로 어떤 상태를 먼저 사용할 지 우선순위를 정하는 메커니즘이 필요하다.
    - 디코더가 모든 디코딩 타임스텝마다 인코더의 각 상태에 다른 가중치 또는 '어텐션'을 할당한다.
    - 아래 그림에서 어텐션은 출력 시퀸스에 있는 두 번째 토큰을 예측하는 역활을 한다.

        ![Attention_Mechanism](image/01_02_attention_mechanism.png)

2. 어텐션 기반 모델은 타임스텝마다 가장 많이 관련된 입력 토큰에 초점을 맞춘다.
    - 문장에 있는 단어와 원 문장에 있는 단어의 복잡한 정렬 문제를 학습한다.
    - 영어-프랑스어 번역 모델의 어텐션 가중치가 아래 이미지와 같다고 가정한다.
        - 각 픽셀은 가중치를 나타낸다.
        - 'zone'과 동일한 다섯 번째 토큰은 'European'이지만 일곱 번째 토큰인 'Area"와의 가중치가 높은(밝은 색)을 가진다.

            ![Attention_Weights](image/01_02_attention_weights.png)

3. RNN은 태생적으로 계산이 순차적으로 수행되며 입력 시퀸스 전체에 걸쳐 병렬화할 수 없다.

4. 트랜스포머는 순환을 없애고 셀프 어텐션이라는 특별한 형태의 어텐션에 전적으로 의존한다.
    - 신경망의 같은 층에 있는 모든 상태에 대해서 어텐션을 계산하는 메커니즘이다.
    - 인코더와 디코더는 각각 셀프 어텐션이 있다.
    - 어텐션의 출력은 피드 포워드 신경만에 주입된다.

        ![Transformer_Architecture](image/01_transformer_architecture.png)

## Chapter 3 NLP의 전이 학습 <a class="anchor" id="chapter3"></a>
1. 컴퓨터 비전에서는 전이 학습을 사용해 ResNet 같은 합성곱 신경망을 한 작업에서 훈련한 다음 새로운 작업에 적용하거나 미세 튜닝을 하는 경우가 많다.
    - 이를 통해서 신경망은 원래 작업에서 학습한 지식을 사용한다.
    - 구조적으 모델은 바디와 헤드로 구성된다.
    - 바디의 가중치는 훈련하는 동안 원래 도메인에서 다양한 특성을 학습하고, 이 가중치를 사용해 새로운 작업을 위한 모델을 초기화한다.
    - 전통적인 지도 학습과 비교하면 전이 학습은 일반적으로 다양한 작업에서 적은 양의 레이블 데이터로 훨씬 효과적으로 훈련하는 높은 품질의 모델을 만든다.
    
        ![Transfer_Learning_Computer_Vision](image/01_03_transfer_learning_computer_vision.png) 

2. 컴퓨터 비전에서는 먼저 이미지가 수백만 개인 ImageNet 같은 데이터셋에서 모델을 훈련한다.
    - 이 과정은 사전훈련이며, 모델에게 에지나 색깔 같은 기본적인 이미지 특징을 학습시킨다.

3. 그런 다음 사전 훈련된 모델을 후속 작업에서 미세 튜닝할 수 있다.
    - 예를 들면 비교적 적은 개수의 레이블링된 샘를로 꽃 품종을 분류할 수 있다.

4. 미세 튜닝한 모델은 일반적으로 처음부터 동일한 양의 레이블링된 데이터에서 훈련하는 지도할 학습 모델보다 정확도가 더 높다.

5. NLP에서 전이 학습을 위한 범용적인 프레임워크 ULMFiT(Universal Language Model Fine-tuning for Text Classification)가 2018년에 소개되었다.
    - ULMFiT는 언어 모델을 사전 훈련한 다음 특정 작업에 미세 튜닝하는 세 단계 프로세스를 따른다.
    - 사전 훈련
        - 이전 단어를 바탕으로 다음 단어를 예측하는 것을 목표로하며, 이 작업을 언어 모델링이라고한다.
        - 레이블링된 데이터가 필요하지 않으며 위키드피아 같은 소스에 있는 풍부한 텍스트를 활용한다.
    - 도메인 적응
        - 언어 모델을 대규모 말붕치에 사전 훈련 한 후, 도메인 내 말뭉치에 적응시킨다.
        - 아래의 그림과 같이 위키피아에서 훈련한 모델을 IMDb 영화 리뷰 말뭉치에 적용한다.
        - 여전히 언어모델링을 사용하지만 모델은 IMDB 영화리뷰에 있는 다음 단어를 예측한다.
    - 미세 튜닝
        - 모델을 타깃 작업을 위한 분류 층과 함께 미세튜닝한다.

            ![ULMFiT_Process](image/01_03_ulmfit_process.png)

6. GPT
    - 트랜스포머 아키텍처의 디코더 부분만 사용하고 ULMFiT와 유사한 전이 학습 접근법을 따른다.

7. BERT
    - 트랜스포머 아키텍처의 인코더 부분을 사용하고 마스크드 언어 모델링이라는 특별한 형태의 언어 모델링을 사용한다.
    - 마스크드 언어 모델링의 목표는 문장에서 무작위로 선택한 단어를 마스킹하고 모델이 이 마스크된 단어를 예측하도록 훈련하는 것이다.
       - 예를 들어 "The cat sat on the mat"라는 문장에서 "cat"과 "mat"을 마스킹하면 모델은 "The [MASK] sat on the [MASK]"에서 마스크된 단어를 예측해야 한다.

8. GPT와 BERT는 서로 호환되지 않는 프레임워크(파이토치, 텐서플로우)를 사용해 모델을 릴리스 했다.
    - NLP 기술자들이 자신의 애플리케이션에 포팅하기 어렵게 만들었다.

9. 허깅페이스 트랜스포머가 릴리스되면서 점진적으로 50여 개의 아키텍처에 걸쳐 단일화된 API가 구축되었다.


## Chapter 4 허깅페이스 트랜스포머스 <a class="anchor" id="chapter4"></a>
1. 새로운 머신러닝 아키텍처를 작업에 적용하는 것은 일반적으로 다음과 같은 단계를 거친다.
    - 모델 아키텍척를 코드로 구현한다. 파이토치 또는 텐서플로를 사용한다.
    - 서버로쿠터 사전 훈련된 가중치를 로드한다.
    - 입력을 전처리하고 모델에 전달한다. 그다음 작업에 맞는 상후 처리를 수행한다.
    - 데이터로더를 구현하고 모델 훈련을 위한 손실 함수와 옵티마이어를 정의한다.

2. 모델과 작업에 따라 단계마다 별도의 사용자 정의 로직이 필요하다.
    - 관행적으로 연구실은 논문을 발표할 때 코드와 모델 가중치도 함께 릴리즈한다.
    - 그러나 코드는 거의 표준화가 안 돼 있어서 새로운 문제에 적용하기 위해 수정하는데 며칠이 걸릴 수 있다.   

3. 허깅페이스 트랜스포머스는 이러한 문제를 해결하기 위해 만들어졌다.
    - 파이토치와 텐서플로우 모두에서 작동하는 단일화된 API를 제공한다.
    - 수백 개의 사전 훈련된 모델과 토크나이저를 제공한다.
    - 모델을 특정 작업에 맞게 미세 튜닝하는 데 도움이 되는 유틸리티도 포함되어 있다.
    - 허깅페이스 트랜스포머스를 사용하면 몇 줄의 코드로 복잡한 NLP 작업을 수행할 수 있다.

## Chapter 5 트랜스포머 애플리케이션 둘러보기 <a class="anchor" id="chapter5"></a>
1. 아래는 온라인 주문에 대한 피드백이다.
    - 고객 피드백이 긍정적인지 부정적인지 감성분석을 수행해본다.


In [4]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure 
from your online store in Germany. Unfortunately, when I opened the package, 
I discovered to my horror that I had been sent an action figure of Megatron 
instead! As a lifelong enemy of the Decepticons, I hope you can understand my 
dilemma. To resolve the issue, I demand an exchange of Megatron for the 
Optimus Prime figure I ordered. Enclosed are copies of my records concerning 
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

2. 트랜스포머는 다양한 추상화 수준에서 라이브러와 상호작용하도록 계층화된 API를 제공한다.
    - 원시 텍스트를 미세 튜닝된 모델의 예측으로 변환하기 위해 필요한 모든 단계를 추상화하는 파이프라인을 사용한다.

In [2]:
# 파이프라인 객체 생성
#   - 허깅페이스 허브에서 모델 가중치를 다운로드하고, 다음 다운로드 시에는 캐싱된 모델을 사용한다.
#   - text-classification 파이프라인은 감성분석에 사용된다.
from transformers import pipeline
classifier = pipeline("text-classification")

/home/redinblue/anaconda3/envs/pybuild/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-25 11:31:20.100995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766629880.179588    1174 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766629880.203281    1174 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766629880.364468    1174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

In [ ]:
# 예측데이터 생성
#   - 판다스의 DataFrame을 사용하여 텍스트 데이터를 생성한다.
import pandas as pd
outputs = classifier(text)
df = pd.DataFrame(outputs)

# 모델은 텍스트가 부정적이라고 확신한다.
print(df)

      label     score
0  NEGATIVE  0.901546


3. NLP에서는 제품, 장성, 사람 같은 실제 객체를 개체명(Named Entity)이라고 하고 이런 개체명을 추출하는 작업을 개체명 인식(Named Entity Recognition, NER)이라고 한다.
    - 예를 들어 "Hugging Face is a company based in New York City"라는 문장에서 "Hugging Face"는 조직, "New York City"는 위치로 태그된다.
    - NER은 정보 추출, 질문 답변, 문서 요약 같은 다양한 NLP 작업에서 중요한 역활을 한다.

In [6]:
# NRE(개체명 인식)에 고객 피드백 적용
#   - aggregation_strategy="simple" 옵션을 사용하여 개체명을 단일 엔티티로 그룹화한다.
#   - 예를 들어 "New"와 "York"가 별도의 토큰으로 태그되는 대신 "New York"가 단일 위치 엔티티로 태그된다.
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
df = pd.DataFrame(outputs)
print(df)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


  entity_group     score           word  start  end
0          ORG  0.879010         Amazon      5   11
1         MISC  0.990859  Optimus Prime     36   49
2          LOC  0.999755        Germany     91   98
3         MISC  0.556568           Mega    210  214
4          PER  0.590257         ##tron    214  218
5          ORG  0.669691         Decept    256  262
6         MISC  0.498350        ##icons    262  267
7         MISC  0.775361       Megatron    354  362
8         MISC  0.987854  Optimus Prime    372  385
9          PER  0.812096      Bumblebee    508  517


4. 모든 개체명을 감지하고 ORG(조직), LOC(위치), PER(사람), MISC(기타) 같은 NER 카테고레이 할당했다.
    - 점수는 모델이 개체명을 얼마나 확신하는지 나타낸다.
    - "Decepticons"과 "Megatron"은 확신이 낮아서 두 경우 모두 하나의 개체로 그룹화되지 않았다.

5. 질문 답변에서는 텍스트 구절과 함께 얻고 싶은 질문을 모델에 전달하고, 모델은 답변 텍스트를 반환한다.
    - 아래의 예제의 경우 답변을 텍스트에서 직접 추출하기 때문에 추출적 질문 답변이라고 한다.
    - 고객 피드백에서 관련 정보를 빠르게 추출한다.


In [9]:
# 고객 피드백에 대한 질문에 대한 답변 생성
#   - question-answering 파이프라인은 질문 답변에 사용된다.
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
df = pd.DataFrame([outputs])
print(df)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


      score  start  end                   answer
0  0.631292    339  362  an exchange of Megatron


6. 텍스트 요약의 목표는 긴 텍스트를 입력으로 받고 관련 사실이 포함된 간한 버전을 생성한다.
    - 모델이 논리적인 텍스트를 생성해야 하므로 이전 문제보다 훨씬 더 복잡한 작업니다.
    = "Bumblebee"가 불만을 제기한 사람이라는 것을 정확하기 구분했다.

In [10]:
# 고객 피드백에 대한 요약생성
#   - summarization 파이프라인은 텍스트 요약에 사용된다.
#   - max_length 파라미터는 생성된 요약의 최대 길이를 지정한다.
#   - clean_up_tokenization_spaces 파라미터는 토크나이저로 인해 발생하는 불필요한 공백을 제거한다.
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Your min_length=56 must be inferior than your max_length=45.
/home/redinblue/anaconda3/envs/pybuild/lib/python3.10/site-packages/transformers/generation/utils.py:1633: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when he opened the package, he discovered to his horror that he had been sent an action figure of Megatron instead.


7. 번역 또한 텍스트를 생성해 출력하는 작업니다.

In [5]:
# 번역 파이브라인을 사용하여 영어를 독일어로 번역
#   - translation_en_to_de 파이프라인은 영어에서 독일어로 번역에 사용된다.
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Device set to use cuda:0


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.


8. 자동 완성 기능으로 고객 피드백을 빠르게 응답하고 싶은 경우 텍스트 생성 모델을 사용한다.

In [6]:
# 고객 피드백에 대한 응답 생성
#   - text-generation 파이프라인은 텍스트 생성에 사용된다.
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order  was mixed up."
prompt = text + "\n\nCustomer Service Response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dear Amazon, last week I ordered an Optimus Prime action figure 
from your online store in Germany. Unfortunately, when I opened the package, 
I discovered to my horror that I had been sent an action figure of Megatron 
instead! As a lifelong enemy of the Decepticons, I hope you can understand my 
dilemma. To resolve the issue, I demand an exchange of Megatron for the 
Optimus Prime figure I ordered. Enclosed are copies of my records concerning 
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer Service Response:
Dear Bumblebee, I am sorry to hear that your order  was mixed up. I

did not receive your order nor did I receive the product.

This is a simple error. I received an order for Megatron action figure from you online store in Germany and did not receive the package or a

product. As such, I was unable to process the order.

I have contacted you to inform you that that is not the case.

I have also sent a letter to you from your store. Please understan

## Chapter 6 허깅페이스 생태계 <a class="anchor" id="chapter6"></a>
1. 트랜스포머스는 NLP와 머신거닝 프로젝트의 속도를 높이는 다양한 라이브러리 도구를 갖춘 생태계로 빠르게 선장했다.
    - 라이브러리와 허크로 구성된다.
    - 라이브러리는 코드를 제고하고, 허브는 사전 훈련된 모델 가중치, 데이터셋, 평가지표를 위한 스크립트를 제공한다.

    ![Huggingface_Ecosystem](image/01_06_huggingface_ecosystem.png)

2. 허깅페이스 허브
    - 무료로 공개된 20,000개 이상의 모델을 호스팅한다.
    - 아래의 그림과 같이 작업, 프레임워크, 데이터셋 등으로 필터링해 허브를 탐색하면 가능성 있는 후보 모델을 빠르게 찾을 수 있다.
    - 허브는 모델 가중치 외에 데이터셋과 평가 지표 계산을 위한 스크립트도 호스팅하고 잇어 공개된 결과를 재할 수 있다.
    - 모델과 데이터셋 내용을 문서와한 모델 카드와 데이터셋 카드도 제공하므로 사용자는 자신에게 적합한지 판단할 수 있다. 

        ![Huggingface_Model_Hub](image/01_06_huggingface_model_hub.png)

    - 아래의 그림처럼 위젯을 통해 모델일 직접 테스트하는 것도 가능하다.

        ![Huggingface_Model_Widget](image/01_06_huggingface_model_widget.png)

3. 파이토치와 텐서플러도 각각 허브를 제공하므로 허깅페이스 허브에 특정 모델이나 데이터셋이 없는 경우 이들 허브에서 찾아볼 수 있다.

4. 허깅페이스 토크나이져
    - 트랜스포머 모델은 토큰의 수치 표현에서 훈련한다.
    - 토크나아져는 다양한 토큰화 전략을 제공하는데, 러스트 백앤드 덕분에 매우 빠르게 텍스트를 토큰화 한다.
    - 입력을 정규화 하거나 모델 출력을 적절한 포멧으로 변환하는 등의 모든 전처리, 사후처리 단계를 처리한다.
    - 사전 훈련되 모델 가중치를 로드하는 방식으로 토크나이저를 로드할 수 있다.

5. 허깅페이스 데이터셋
    - 데이터를 다운로드하고 표준 포맷으로 변환하는 일련의 과정을 표준 인터페이스를 제공해 단순화한다.
    - 스마트한 캐싱을 제공하고 메모리 매핑이라는 기법을 사용해 대규모 데이터셋도 메모리에 올리지 않고 효율적으로 처리할 수 있다.
    - 판다스와 넘파이 같은 인기 있는 데이터 처리 라이브러리와도 호환된다.
    - 데이터셋은 많은 평가 지표를 위한 스크립트를 제송해 실험의 재현 가능성과 결과릐 신뢰성을 높인다.

6. 허깅페이스 액셀러레이트
    - 사용자 정의 로직을 처리하는 일반적인 훈련 루프에 훈련 인프라에 필요한 추상화 층을 추가한다.
    - 단일 GPU, 다중 GPU, TPU에서 모두 작동하는 코드를 작성할 수 있다.

7. 트랜스포머의 주요 도전 과제
    - 언어: 연구에 사용된 언어는 거의 영어다. 다른 언어에 적용하려면 추가적인 노력이 필요하다.
    - 데이터 가용성: 대규모 사전 훈련된 모델을 미세 튜닝하려면 레이블링된 데이터가 필요하다. 많은 언어와 도메인에서 레이블링된 데이터가 부족하다.
    - 긴 문서 처리하기: 셀프 어텐션은 텍스트 길이가 문단 정도 될 때 잘 작동한다.
    - 불투명성: 모델이 그렇게 예측한 이유를 설명하기 어렵다. 모델로 중용한 결정을 내릴 때 문제가 될 수 있다.
    - 편향: 주로 인터넷 텍스트로 훈련된 모델은 종종 편향된 출력을 생성한다. 인종차별, 성차멸 등이 포함될 수 있다.